In [ ]:
import pymysql
import pandas as pd
import numpy as np
import time
import datetime
import os
from sqlalchemy import create_engine  
import pprint

In [61]:
db_info = {'user':'bo.ma',  
            'password':'123456',  
            'host':'192.168.100.141',  
            'database':'siyecap'  # 这里我们事先指定了数据库，后续操作只需要表即可
            } 

# 进行查询的语句
engine = create_engine('mysql+pymysql://%(user)s:%(password)s@%(host)s/%(database)s?charset=utf8' % db_info,encoding='utf-8') 
#这里直接使用pymysql连接,echo=True，会显示在加载数据库所执行的SQL语句。
data_path = r'F:\bma\project\data'
startdate = '2007-01-01'
enddate = '2019-12-01'
data_name = 'open,close,high,low,tradingstatus,volume,turnoverrate,volumeyuan'
sql = '''SELECT symbol,date,%s FROM siyecap.stockprice where date between '%s' and '%s';'''%(data_name,startdate,enddate)
datasize = (3141,3824)

In [ ]:
def get_data(data_name,table = 'stockprice',engine=engine):
    sql = '''SELECT symbol,date,%s FROM siyecap.%s where date between '%s' and '%s';'''%(data_name,table,startdate,enddate)
    data = pd.read_sql_query(sql,con = engine)
    data = data.sort_values(by = ['date','symbol'])
    data.index = range(len(data.index))
    widedata = data[['symbol','date',data_name]].pivot_table(columns = ['symbol'], index= ['date'],values=[data_name],dropna= False)
    widedata.columns = widedata.columns.droplevel()
    return widedata

## 提取行情数据，整合、处理

In [ ]:
def get_tradedays():
    # 返回的为 dataframe 时间为2007/01/01 - 2019/12/01 
    tradeday = pd.read_csv(os.path.join(data_path,'tradeday.csv'),date_parser=['date'])
    print('there are %s tradedays' % len(tradeday))
    return tradeday

def get_tickers():
    tickers = pd.read_csv(os.path.join(data_path,'tickers.csv'))
    print('there are %s stocks' % len(tickers))
    return tickers

# 标准的大小 3141 tradeday 3824 stocks
tradeday = get_tradedays()
tickers = get_tickers()

In [ ]:
def save_data(data_name_list,startdate =  '2007-01-01',enddate = '2019-12-01',engine = engine): 
    data_str = ','.join(data_name_list)
    sql = '''SELECT symbol,date,%s FROM siyecap.stockprice where date between '%s' and '%s';'''%(data_str,startdate,enddate)
    time1 = time.time()
    data = pd.read_sql_query(sql,con = engine)
    data = data.sort_values(by = ['date','symbol'])
    data.index = range(len(data.index))
    for data_name in data_name_list:
        time10 = time.time()
        widedata = data[['symbol','date',data_name]].pivot_table(columns = ['symbol'], index= ['date'],values=[data_name],dropna= False)
        widedata.columns = widedata.columns.droplevel()
        widedata.to_csv(os.path.join(data_path,'%s.csv')%data_name)
        time11 = time.time()
        print('getting and reshaping %s data cost %.2f s'%(data_name,time11-time10))
    time2 = time.time()
    print('getting and reshaping all data cost %.2f s'%(time2-time1))

In [ ]:
%%time
data_name_list = ['tradingstatus','close','open','high','low','avgprice','volume','adjustfactor','volumeyuan','turnoverrate',
                  'pe','pe_ttm','ps','ps_ttm','pb','est_eps']
save_data(data_name_list)

In [ ]:
test_close =  pd.read_csv(os.path.join(data_path,'close'+'.csv'),index_col= 0)

In [ ]:
close = pd.read_csv(os.path.join(data_path,'close.csv'),index_col=0)
adjustfactor = pd.read_csv(os.path.join(data_path,'adjustfactor.csv'),index_col=0)
close_re = close*adjustfactor

In [ ]:
# 将原始数据处理后复权的数据 可以进行前后对比
data_path = r'F:\bma\project\data'
data_list = ['close', 'open', 'high', 'low']

data_dict = {}
adjustfactor = pd.read_csv(os.path.join(data_path,'adjustfactor.csv'),index_col=0)
for data_name in data_list:
    tempdata = pd.read_csv(os.path.join(data_path,data_name+'_raw.csv'),index_col=0)
    tempdata = tempdata * adjustfactor
    tempdata.to_csv(os.path.join(data_path,data_name+'.csv'))

提取指数数据

In [ ]:
sql = '''SELECT symbol,date,%s FROM siyecap.indexprice where date between '%s' and '%s';'''%(data_name,startdate,enddate)
data = pd.read_sql_query(sql,con = engine)

In [ ]:
data = data.sort_values(by = ['date','symbol'])
data.index = range(len(data.index))

In [ ]:
index_name = ['000001.SH','000300.SH','399001.SH','000016.SH']

In [ ]:
data_name = 'close'
time10 = time.time()
widedata = data[['symbol','date',data_name]].pivot_table(columns = ['symbol'], index= ['date'],values=[data_name],dropna= False)
widedata.columns = widedata.columns.droplevel()
# widedata.to_csv(os.path.join(data_path,'%s.csv')%data_name)
time11 = time.time()

In [ ]:
widedata.to_csv(os.path.join(data_path,'index_%s.csv')%data_name)

获取涨跌停--done

In [ ]:
sql = '''SELECT symbol,date,%s FROM siyecap.stockinfo where date = '2019-10-10';'''%('ipodays') # stockinfo 里面没有科创板
data = pd.read_sql_query(sql,con = engine)

In [ ]:
# 提取科创板的list 目前先不做科创板 做一个过滤科创版的filter
tickers  = pd.read_csv(os.path.join(data_path,'tickers.csv'))
tickers2 = data['symbol']

In [ ]:
tickers_kechuang = [x for x in  tickers.values.T[0] if x[:3] == '688' ]

In [ ]:
# 去掉科创板后，涨跌幅的限制就比较处理，只要涨跌幅超过9.5 就认为无法买入；科创板有5天的限制，五天内涨跌幅不受限。
tickers_kechuang = pd.DataFrame({'symbol':tickers_kechuang})
tickers_kechuang.to_csv(os.path.join(data_path,'kechuangban_list.csv'))

In [ ]:
tickers_kechuang2 = [x for x in  list(set(tickers.values.T[0]) - set(tickers2.values.T[0])) if x[:3] == '688' ]   

获取每天的涨跌幅

In [ ]:
changepct = get_data('changepct')

In [24]:
changepct.to_csv(os.path.join(data_path,'changepct.csv'))

涨跌停-利用high，low以及volumeyuan计算

In [26]:
high = pd.read_csv(os.path.join(data_path,'high.csv'),index_col=0)
low = pd.read_csv(os.path.join(data_path,'low.csv'),index_col=0)
volumeyuan = pd.read_csv(os.path.join(data_path,'volumeyuan.csv'),index_col=0)
changepct = pd.read_csv(os.path.join(data_path,'changepct.csv'),index_col=0)

In [58]:
# 这里设置的是一字涨停或者一字跌停 ；盘中或者盘尾也是可以建仓的
upperLimit = ((high == low) & (changepct > 0)) 
downLimit = ((high == low) & (changepct < 0)) 

upperLimit.to_csv(os.path.join(data_path,'upperLimit.csv'))
downLimit.to_csv(os.path.join(data_path,'downLimit.csv'))

获取行业分类--done

In [ ]:
sql = '''SELECT symbol,date,%s FROM siyecap.%s where date between '%s' and '%s';'''%('quant1level1','citicscomponent',startdate,enddate) # stockinfo 里面没有科创板
data = pd.read_sql_query(sql,con = engine)

In [ ]:
data = data.sort_values(by = ['date','symbol'])
data.index = range(len(data.index))

In [ ]:
widedata = data.pivot(columns = 'symbol', index= 'date')
widedata.columns = widedata.columns.droplevel()
widedata.index = [ str(x.date()) for x in widedata.index]
widedata = widedata.reindex(index = list(tradeday.values.T[0]),columns=list(tickers.values.T[0]))
widedata[(widedata == '')] = np.nan
widedata.to_csv(os.path.join(data_path,'quant1level1.csv'))

In [ ]:
quantlevel_list = pd.read_csv(os.path.join(data_path,'quantlevelname.csv'),encoding='gbk')

In [ ]:
def get_industry_dummy():
    quantlevel_list = pd.read_csv(os.path.join(data_path,'quantlevelname.csv'),encoding='gbk')
    qauntlevel = pd.read_csv(os.path.join(data_path,'quant1level1.csv'),index_col=0,low_memory=False)
    industry_dummy = {}
    for industry in quantlevel_list.iloc[:,0]:
        print(f'{industry} generate done!')
        industry_dummy.update({industry:qauntlevel == industry})
    return industry_dummy

In [ ]:
get_industry_dummy()

获取停牌数据

In [ ]:
volumeyuan = pd.read_csv(os.path.join(data_path,'volumeyuan.csv'),index_col=0)
suspendFlag = volumeyuan ==0
suspendFlag.to_csv(os.path.join(data_path,'volumeyuan.csv'))

In [ ]:
suspendFlag = pd.read_csv(os.path.join(data_path,'suspend.csv'))

获取ST数据

In [59]:
STFlag = get_data('isST','stockinfo')

In [60]:
# 不同表的数据大小不一样需要进行reindex 
STFlag

symbol,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,603988.SH,603989.SH,603990.SH,603991.SH,603992.SH,603993.SH,603996.SH,603997.SH,603998.SH,603999.SH
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2019-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2019-11-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [77]:
STFlag.index = [str(x) for x in list(STFlag.index)]

In [80]:
STFlag = STFlag.reindex(index = list(tradeday.date) ,columns=list(tickers.symbol))

In [81]:
STFlag.to_csv(os.path.join(data_path,'ST.csv'))